In [11]:
import main

In [12]:
root

'/Users/andrewcheng/Documents/GitHub/Debuggers/'

In [1]:
import torch
import os
import pandas as pd
import numpy as np
import BugNIST_metric
import segment_mixture
import tifffile
from VolumesDataset import VolumesDataset

# CNN
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
import torchvision
from torchvision import transforms

ModuleNotFoundError: No module named 'segment_mixture'

In [2]:
root = os.path.abspath('../') + '/'
root

'/Users/andrewcheng/Documents/GitHub/Debuggers/'

In [3]:
example_train_fp = root + "BugNIST_DATA/train/AC/bcrick_10_000.tif"
example_valid_fp = root + 'BugNIST_DATA/validation/mix_02_006.tif'
example_valid_gen_fps = [root + 'temp/mix_02_006/bug_' + str(i) + '.tif' for i in range(13)]

In [4]:
example_custom_train_fp = root + "BugNIST_DATA/custom_train/AC/bcrick_10_000.tif"

In [5]:
import re
import os

import plotly.graph_objects as go
import plotly.io as pio

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import napari

In [6]:
bug1 = io.imread(example_custom_train_fp)
bug2 = io.imread(example_train_fp)

In [7]:
# View the bug with napari
viewer = napari.view_image(bug1)

In [8]:
# View the bug with napari
viewer = napari.view_image(bug2)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *

In [5]:
# define the 3D CNN
num_classes = 12

# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(3, 32)
        self.conv_layer2 = self._conv_layer_set(32, 64)
        self.fc1 = nn.Linear(376320, 64)
        self.relu = nn.LeakyReLU()
        self.drop=nn.Dropout(p=0.50)
        self.classifier = nn.Linear(64, num_classes)
        
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.drop(out)
        out = self.conv_layer2(out)
        out = self.drop(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.classifier(out)
        
        return out

#Definition of hyperparameters
n_iters = 100
num_epochs = 50

# Create CNN
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel() #.to(device)

#model.cuda()
print(model)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.04
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=376320, out_features=64, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (drop): Dropout(p=0.5, inplace=False)
  (classifier): Linear(in_features=64, out_features=12, bias=True)
)


In [6]:
model_path = root + 'Notebooks/models'
model = torch.load(model_path)

In [7]:
int_to_bug = {0: 'ac',
 1: 'bc',
 2: 'bf',
 3: 'bl',
 4: 'bp',
 5: 'cf',
 6: 'gh',
 7: 'ma',
 8: 'ml',
 9: 'pp',
 10: 'sl',
 11: 'wo'}

In [8]:
transform_with_unsqueeze = transforms.Compose([
    transforms.Lambda(lambda img: torch.from_numpy(img)),  # Convert to PyTorch tensor
    transforms.Lambda(lambda img: torch.unsqueeze(img, dim=0).repeat(3, 1, 1, 1)) # Add more transforms as needed
])

In [9]:
tifffile.imread(example_valid_gen_fps[4]).shape

(128, 92, 92)

In [10]:
# Make single bug prediciton
def predict_bugs(filepaths):
 
    # read it in
    # cast into tensor
    data = pd.DataFrame({'FileLoc' : filepaths, 'BugType' : ['bl'] * len(filepaths)})
    dataset = VolumesDataset(data, transform=transform_with_unsqueeze)    
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)
    

    # Predict
    preds = []
    
    for img, label in data_loader:
        with torch.no_grad():
            out = model(img)
            pred = torch.argmax(out)
            preds.append(int_to_bug[int(pred)])
    
    return preds

In [11]:
predict_bugs(example_valid_gen_fps)

['bf', 'bc', 'ml', 'bc', 'bc', 'bc', 'bf', 'bc', 'bf', 'ma', 'ml', 'sl', 'wo']

In [12]:
# Predict Validation
def predict_validation(filepath):
    
    res = []
    
    watershed_centers, segment_paths = segment_mixture.segment_bugs(filepath)
    segment_pred = predict_bugs(segment_paths)
    
    # Segment
    for watershed_center, pred in zip(watershed_centers, segment_pred):
        x, y, z = watershed_center
        
        res.append(pred + ';' + str(x) + ';' + str(y) + ';' + str(z))
    
    return ';'.join(res)
    

In [13]:
predict_validation(example_valid_fp)

'bf;61;45;47;bc;39;47;79;ml;95;38;47;bc;58;71;57;bc;51;29;22;bc;60;14;41;bf;65;19;29;bc;41;61;75;bf;58;47;76;ma;50;42;48;ml;38;63;20;sl;46;56;29;wo;58;41;29'

In [14]:
# With File Paths
bug_labels = {'AC' : 'Brown Cricket', 'BC' : 'Black Cricket', 'BF' : 'Blow fly', 
              'BL' : 'Buffalo Beetle Larva' , 'BP' : 'Blow Fly Pupa', 'CF' : 'Curly-wing Fly', 'GH' : 'Grasshopper',
              'MA' : 'Maggot', 'ML' : 'Mealworm', 'PP' : 'Green Bottle Fly Pupa' , 'SL' : 'Soldier Fly Larva',
              'WO' : 'Woodlice'}

In [15]:
# Train Data

train_fp = root + 'BugNIST_DATA/train/'

A = []
train_file_paths = []

for i, v in bug_labels.items():
    for fp in os.listdir(train_fp + i):
        train_file_paths.append(train_fp + i + '/' + fp)
        A.append({'fp' : train_fp + i + '/' + fp, 'type' : v})
        
train = pd.DataFrame(A)
train.head()

,fp,type
0,/Users/andrewcheng/Documents/GitHub/Debuggers/...,Brown Cricket
1,/Users/andrewcheng/Documents/GitHub/Debuggers/...,Brown Cricket
2,/Users/andrewcheng/Documents/GitHub/Debuggers/...,Brown Cricket
3,/Users/andrewcheng/Documents/GitHub/Debuggers/...,Brown Cricket
4,/Users/andrewcheng/Documents/GitHub/Debuggers/...,Brown Cricket


In [16]:
train.shape

(9154, 2)

In [17]:
# Validation Data
validation = pd.read_csv(root + 'BugNIST_DATA/validation/validation.csv')
validation.head()

,filename,centerpoints
0,mix_02_006.tif,bl;48.61;39.14;96.19;bl;77.71;53.14;57.17;pp;5...
1,mix_02_011.tif,ml;77.86;46.26;57.79;sl;41.14;61.93;32.37;sl;5...
2,mix_03_001.tif,ml;54.20;34.58;81.43;ml;59.60;41.94;60.72;ml;4...
3,mix_03_003.tif,wo;44.37;27.71;85.18;wo;30.04;28.46;58.76;wo;4...
4,mix_03_004.tif,pp;74.65;44.18;52.53;pp;30.69;25.53;54.51;pp;1...


In [18]:
validation.shape

(78, 2)

In [19]:
def run_pipeline(out_path = 'baseline.csv'):
    
    validation_fp = root + 'BugNIST_DATA/validation/'

    validation = pd.read_csv(validation_fp + 'validation.csv')
    validation['centerpoints'] = validation['filename'].apply(lambda x : predict_validation(validation_fp + x))
    
    # Save predictions to csv
    validation.set_index('filename').to_csv(out_path)

In [20]:
run_pipeline()

In [21]:
pred = root + 'Notebooks/baseline.csv'
target = root + 'BugNIST_DATA/validation/validation.csv'

In [22]:
BugNIST_metric.main(pred, target)

F1 score: 0.06521705612220399
